In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
with open('X.npy', 'rb') as f:
  X = np.load(f)

with open('y.npy', 'rb') as f:
  y = np.load(f)

In [ ]:
X.shape, y.shape

((1460, 60), (1460, 1))

In [ ]:
x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(X)
scaled_X = x_min_max_scaler.transform(X)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(y)
scaled_y = y_min_max_scaler.transform(y)

In [ ]:
# K-fold cross validation
K = 10
kf = KFold(n_splits=K)

In [ ]:
rmses = []
for train_index, test_index in kf.split(scaled_X):
  scaled_X_train, scaled_X_test = scaled_X[train_index], scaled_X[test_index]
  scaled_y_train, scaled_y_test = scaled_y[train_index], scaled_y[test_index]
  y_test = y[test_index]

  # training
  model = keras.Sequential(
      [
        keras.Input(shape=scaled_X_train.shape[-1]),
        layers.Dense(96, activation='relu'),
        layers.Dense(48, activation='relu'),
        layers.Dense(1)
      ]
  )

  # model.compile(loss="mse", optimizer="adam")

  opt = keras.optimizers.Adam(learning_rate=0.005)
  model.compile(loss="mse", optimizer=opt)

  early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)
  model = model.fit(scaled_X_train, scaled_y_train,
                    batch_size=1, epochs=150,
                    callbacks=[early_stopping_callback], validation_split=0.05, verbose='auto')

  # evaluation
  pred = model.predict(scaled_X_test).reshape((-1, 1))
  pred = y_min_max_scaler.inverse_transform(pred)
  rmse = np.sqrt(metrics.mean_squared_error(y_test, pred))

  print(rmse)
  print("--------------")
  

  rmses.append(rmse)

print("average rmse:", np.mean(rmses))

# Hyperparameter
- batch size
- learning rate
- number of layers
- feature sizes
- activation functions
- optimizers

# Grid Search

In [ ]:
batch_sizes = np.arange(1, 10, 2) # batch size는 1에서부터 10까지 2칸씩 띄어가면서 탐색하고 싶음.

In [ ]:
batch_sizes

array([1, 3, 5, 7, 9])

In [ ]:
learning_rates = [0.005, 0.01, 0.02]  # learning rate는 0.005, 0.01, 0.02를 탐색할 것

In [ ]:
# 총 15개의 세트가 나온다.
# 여기서 최적의 rmse값이 나오는 hyperparameter set을 골라내면 hyperparameter tuning을 했다고 할 수 있다.
for batch_size in batch_sizes:
  for learning_rate in learning_rates:
    print("batch_size:", batch_size, "learning_rate:", learning_rate)

batch_size: 1 learning_rate: 0.005
batch_size: 1 learning_rate: 0.01
batch_size: 1 learning_rate: 0.02
batch_size: 3 learning_rate: 0.005
batch_size: 3 learning_rate: 0.01
batch_size: 3 learning_rate: 0.02
batch_size: 5 learning_rate: 0.005
batch_size: 5 learning_rate: 0.01
batch_size: 5 learning_rate: 0.02
batch_size: 7 learning_rate: 0.005
batch_size: 7 learning_rate: 0.01
batch_size: 7 learning_rate: 0.02
batch_size: 9 learning_rate: 0.005
batch_size: 9 learning_rate: 0.01
batch_size: 9 learning_rate: 0.02


In [ ]:
results = []
for batch_size in batch_sizes:
  for learning_rate in learning_rates:
    print(batch_size, learning_rate)
    for train_index, test_index in kf.split(scaled_X):
      scaled_X_train, scaled_X_test = scaled_X[train_index], scaled_X[test_index]
      scaled_y_train, scaled_y_test = scaled_y[train_index], scaled_y[test_index]
      y_test = y[test_index]

      # training
      model = keras.Sequential(
          [
            layers.InputLayer(input_shape=scaled_X_train.shape[-1]),
            layers.Dense(96, activation='relu'),
            layers.Dense(48, activation='relu'),
            layers.Dense(1)
          ]
      )

      # model.compile(loss="mse", optimizer="adam")

      opt = keras.optimizers.Adam(learning_rate=0.005)
      model.compile(loss="mse", optimizer=opt)

      early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)
      model = model.fit(scaled_X_train, scaled_y_train,
                        batch_size=1, epochs=150,
                        callbacks=[early_stopping_callback], validation_split=0.05, verbose='auto')

      # evaluation
      pred = model.predict(scaled_X_test).reshape((-1, 1))
      pred = y_min_max_scaler.inverse_transform(pred)
      rmse = np.sqrt(metrics.mean_squared_error(y_test, pred))

      print(rmse)
      print("--------------")

      result = {}
      result['batch_size'] = batch_size
      result['learning_rate'] = learning_rate
      result['rmse'] = rmse

      results.append(result)

In [ ]:
results

# Random Search

In [ ]:
# grid search에서는 범위를 어떤 룰에 의해 만들었다면, 
# random search는 시작과 끝점만 정해주고, 그 사이에서 랜덤으로 값을 뽑아오게 된다.
import random

In [ ]:
batch_sizes = random.sample(range(1, 11), 5)

In [ ]:
batch_sizes

[8, 4, 7, 5, 10]

In [ ]:
learning_rates = np.random.uniform(low=0.005, high=0.1, size=(4,))

In [ ]:
# 정말 상상도 못했던 숫자에서 최적의 값이 나올 수 있음!
learning_rates

array([0.08396272, 0.02367816, 0.06519007, 0.07035935])